In [1]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging
from tqdm import tqdm

In [2]:
# Set the project name (this is the folder name where the output files will be stored)
custom_data_folder = 'amsterdam_custom_samples'

In [3]:
## Number of TFRecords in a single file
batch_size = 100

In [4]:
serval_data_folder = "../data"

In [15]:
input_selected_classes_filepath = serval_data_folder + '/' + custom_data_folder + '/csv_files/04_input_selected_classes.csv'
output_class_mapping_filepath   = serval_data_folder + '/' + custom_data_folder + '/csv_files/04_output_class_mapping.csv' 
output_tfrecords_train_path     = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/train'
output_tfrecords_eval_path      = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/eval'

tfrecords_train_search_string   = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/train_*.tfrecord'
tfrecords_eval_search_string    = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/eval_*.tfrecord'

In [16]:
print('[INFO]: When proportion_xxx_samples > 1.0 than samples are duplicated')
print('[INFO]: When proportion_xxx_samples < 1.0 than samples are reduced for this class')
print('')
print('[INFO]: Re-run the notebook a couple of times until you get the amounts per class you would like to see')
print('')
df_input_selected_classes = pd.read_csv(input_selected_classes_filepath, sep=";")
print('[INFO]: df_input_selected_classes')
display(df_input_selected_classes)

[INFO]: When proportion_xxx_samples > 1.0 than samples are duplicated
[INFO]: When proportion_xxx_samples < 1.0 than samples are reduced for this class

[INFO]: Re-run the notebook a couple of times until you get the amounts per class you would like to see

[INFO]: df_input_selected_classes


,label,new_label,proportion_train_samples,proportion_eval_samples,description,enable
0,73,0,1.0,1.0,Domestic animals | pets,1
1,307,1,1.0,1.0,Car,1
2,308,2,1.0,1.0,Vehicle horn | car horn | honking,1
3,314,3,1.0,1.0,Car passing by,1
4,316,4,1.0,1.0,Truck,1
5,318,5,1.0,1.0,Air horn | truck horn,1
6,322,6,1.0,1.0,Emergency vehicle,1
7,347,7,1.0,1.0,Chainsaw,1
8,353,8,1.0,1.0,Accelerating | revving | vroom,1
9,396,9,1.0,1.0,Siren,1


In [17]:
## Create class mapping 

## First validate that there are no duplicates in the input file
assert(not any(df_input_selected_classes.label.duplicated())) ##, "df_input_selected_classes.label is expected to have unique labels!")

## Create class mapping
df_output_class_mapping = df_input_selected_classes.loc[df_input_selected_classes.enable == 1].copy(deep=True)

df_output_class_mapping['old_label'] = df_output_class_mapping['label']
df_output_class_mapping['label'] = df_output_class_mapping['new_label']
## TODO: df_output_class_mapping['label'] = list(range(0, len(df_output_class_mapping)))
df_output_class_mapping['total_count_train'] = 0
df_output_class_mapping['expected_count_train'] = 0
df_output_class_mapping['actual_count_train'] = 0

df_output_class_mapping['total_count_eval'] = 0
df_output_class_mapping['expected_count_eval'] = 0
df_output_class_mapping['actual_count_eval'] = 0

del df_output_class_mapping['enable']
del df_output_class_mapping['new_label']

print('[INFO]: df_output_class_mapping')
display(df_output_class_mapping)

[INFO]: df_output_class_mapping


,label,proportion_train_samples,proportion_eval_samples,description,old_label,total_count_train,expected_count_train,actual_count_train,total_count_eval,expected_count_eval,actual_count_eval
0,0,1.0,1.0,Domestic animals | pets,73,0,0,0,0,0,0
1,1,1.0,1.0,Car,307,0,0,0,0,0,0
2,2,1.0,1.0,Vehicle horn | car horn | honking,308,0,0,0,0,0,0
3,3,1.0,1.0,Car passing by,314,0,0,0,0,0,0
4,4,1.0,1.0,Truck,316,0,0,0,0,0,0
5,5,1.0,1.0,Air horn | truck horn,318,0,0,0,0,0,0
6,6,1.0,1.0,Emergency vehicle,322,0,0,0,0,0,0
7,7,1.0,1.0,Chainsaw,347,0,0,0,0,0,0
8,8,1.0,1.0,Accelerating | revving | vroom,353,0,0,0,0,0,0
9,9,1.0,1.0,Siren,396,0,0,0,0,0,0


In [18]:
def createTFRecord(example_old, new_labels, old_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    global_label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=old_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat, "global_labels": global_label_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [19]:
def getNewLabels(df_label_mapping, old_labels):
    return(df_label_mapping.loc[df_label_mapping.old_label.isin(old_labels)].label.values)

In [20]:
def getAllTFRecords(tfrecords_search_string, df_class_mapping, count_type):
    tfrecords = {}
    
    ## Get files
    files = gfile.Glob(tfrecords_search_string)
    print('[INFO]: Processing ', len(list(files)), ' files from directory: ', tfrecords_search_string, sep='')
    
    for file in tqdm(files):
        records = list(tf.python_io.tf_record_iterator(path = file))
        print('[INFO]: Processing: ', len(records), ' tfrecords from file: ', file, sep='')

        for string_record in records:
            old_tfrecord = tf.train.SequenceExample()
            old_tfrecord.ParseFromString(string_record)

            ## Get labels and find for each label the m-code in class_label_indices_old
            old_labels = old_tfrecord.context.feature['labels'].int64_list.value
            new_labels = getNewLabels(df_class_mapping, old_labels)
            if len(new_labels) == 0:
                ## Skip files without valid labels
                continue
                
            new_tfrecord = createTFRecord(old_tfrecord, new_labels, old_labels)

            ## Update count of found labels
            df_class_mapping[count_type] += np.where(df_class_mapping.old_label.isin(old_labels), 1, 0)

            ## Store new record in dict
            tfrecords[len(tfrecords)] = new_tfrecord
            
    return({'tfrecords':tfrecords, 'df_class_mapping':df_class_mapping})

In [21]:
def writeTFRecordsInBatches(df_class_mapping, tfrecords, batch_size, output_directory, expect_count_param, actual_count_param):
    ## Generate random permutation without replace
    ## Generate multiple to oversample samples if needed
    permutations = [random.sample(range(0, len(tfrecords)), len(tfrecords)), 
                    random.sample(range(0, len(tfrecords)), len(tfrecords)),
                    random.sample(range(0, len(tfrecords)), len(tfrecords)),
                    random.sample(range(0, len(tfrecords)), len(tfrecords))]
    
    batch_index = 0
    tfrecords_remain_in_batch = -1
    for permutation in permutations:
        for index in tqdm(permutation):
            if tfrecords_remain_in_batch == 0:
                batch_writer.close()

            if tfrecords_remain_in_batch <= 0:
                ## Create new batch file
                tfrecords_remain_in_batch = batch_size
                batch_index = batch_index + 1
                batch_file_name = str(output_directory + '/sample_' + str(batch_index) + '.tfrecord')
                batch_writer = tf.python_io.TFRecordWriter(batch_file_name)

            # Update total number of selected samples and check if we should add this one
            tfrecord = tfrecords[index]
            global_labels = tfrecord.context.feature['global_labels'].int64_list.value

            # Skip samples with classes that we already have enough
            if not np.any(df_class_mapping.loc[df_class_mapping[expect_count_param] > df_class_mapping[actual_count_param]]
                                          .old_label.isin(global_labels)):
                continue

            ## Update count of found labels
            df_class_mapping[actual_count_param] += np.where(df_class_mapping.old_label.isin(global_labels), 1, 0)

            ## Write to file
            batch_writer.write(tfrecord.SerializeToString())
            tfrecords_remain_in_batch = tfrecords_remain_in_batch - 1

    batch_writer.close()
    return(df_class_mapping)

In [22]:
def processTFRecords(df_output_class_mapping, output_tfrecords_path, tfrecords_search_string, suffix):
    total_count_param      = 'total_count_' + suffix
    expected_count_param   = 'expected_count_' + suffix
    actual_count_param     = 'actual_count_' + suffix
    proportion_count_param = 'proportion_' + suffix + '_samples'

    ## Read all Records
    tfrecords = getAllTFRecords(tfrecords_search_string, df_output_class_mapping.copy(deep=True), total_count_param)

    # Set back to generic data frame
    df_output_class_mapping = tfrecords['df_class_mapping']
    df_output_class_mapping[expected_count_param] = df_output_class_mapping[proportion_count_param] * df_output_class_mapping[total_count_param]

    ## Write batches
    df_output_class_mapping = writeTFRecordsInBatches(df_output_class_mapping, tfrecords['tfrecords'], batch_size, output_tfrecords_path, expected_count_param, actual_count_param)
    return(df_output_class_mapping)

In [23]:
df_output_class_mapping = processTFRecords(df_output_class_mapping, output_tfrecords_train_path, tfrecords_train_search_string, 'train')


  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]: Processing 1 files from directory: ../data/amsterdam_custom_samples/tfrecords_all/train_*.tfrecord
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
[INFO]: Processing: 33875 tfrecords from file: ../data/amsterdam_custom_samples/tfrecords_all/train_20200507.tfrecord


100%|██████████| 33875/33875 [00:21<00:00, 1555.28it/s]


In [24]:
df_output_class_mapping = processTFRecords(df_output_class_mapping, output_tfrecords_eval_path, tfrecords_eval_search_string, 'eval')

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]: Processing 1 files from directory: ../data/amsterdam_custom_samples/tfrecords_all/eval_*.tfrecord
[INFO]: Processing: 34067 tfrecords from file: ../data/amsterdam_custom_samples/tfrecords_all/eval_20200507.tfrecord


100%|██████████| 34067/34067 [00:21<00:00, 1570.49it/s]


In [25]:
display(df_output_class_mapping)

,label,proportion_train_samples,proportion_eval_samples,description,old_label,total_count_train,expected_count_train,actual_count_train,total_count_eval,expected_count_eval,actual_count_eval
0,0,1.0,1.0,Domestic animals | pets,73,786,786.0,786,813,813.0,813
1,1,1.0,1.0,Car,307,3068,3068.0,3068,3254,3254.0,3254
2,2,1.0,1.0,Vehicle horn | car horn | honking,308,1406,1406.0,1406,1458,1458.0,1458
3,3,1.0,1.0,Car passing by,314,3950,3950.0,3950,3817,3817.0,3817
4,4,1.0,1.0,Truck,316,13532,13532.0,13532,13419,13419.0,13419
5,5,1.0,1.0,Air horn | truck horn,318,220,220.0,220,357,357.0,357
6,6,1.0,1.0,Emergency vehicle,322,1359,1359.0,1359,1464,1464.0,1464
7,7,1.0,1.0,Chainsaw,347,2523,2523.0,2523,2561,2561.0,2561
8,8,1.0,1.0,Accelerating | revving | vroom,353,621,621.0,621,711,711.0,711
9,9,1.0,1.0,Siren,396,993,993.0,993,1029,1029.0,1029


In [26]:
## Write new labels and descriptions back to csv_file
df_output_class_mapping['display_name'] = df_output_class_mapping.description
df_output_class_mapping['index'] = df_output_class_mapping.label
df_output_class_mapping.to_csv(output_class_mapping_filepath, sep = ";",index=False) # hk : index False